# Set up


In [ ]:
!python --version

In [ ]:
%pip install torch torchvision tqdm pandas

In [3]:
import os

import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet101_Weights, resnet101
from tqdm import tqdm

In [4]:
def load_dataset(root, batch_size=32):
    torch.manual_seed(42)

    data_augmentation = transforms.Compose(
        [
            transforms.RandomRotation(degrees=15),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(
                brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1
            ),
        ]
    )
    preprocess = transforms.Compose(
        [
            transforms.Resize(size=(224, 224), antialias=True),
            transforms.ToTensor(),
            transforms.Normalize([0.7037, 0.6818, 0.6685], [0.2739, 0.2798, 0.2861]),
        ]
    )

    # ImageFolder
    train_set = ImageFolder(
        f"{root}/train", transform=transforms.Compose([data_augmentation, preprocess])
    )
    valid_set = ImageFolder(f"{root}/val", transform=preprocess)
    test_set = ImageFolder(f"{root}/test", transform=preprocess)

    # DataLoader
    dataloader = {
        "train": DataLoader(train_set, batch_size=batch_size, shuffle=True),
        "valid": DataLoader(valid_set, batch_size=batch_size, shuffle=False),
        "test": DataLoader(test_set, batch_size=batch_size, shuffle=False),
    }

    return dataloader

In [5]:
def train(
    model: nn.Module,
    dataloader: DataLoader,
    num_epochs=20,
    load_path="./pretrained",
    save_path="./results",
    lr=0.001,
    momentum=0.9,
    resume=False,
):
    torch.manual_seed(42)
    os.makedirs("./results", exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    best_acc = 0.0

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    results = []
    start_epoch = 0
    if resume:
        df = pd.read_csv(f"{load_path}/results.csv")
        results = list(df.T.to_dict().values())
        start_epoch = int(results[-1]["epoch"])

        checkpoint = torch.load(
            f"{load_path}/resnet_{start_epoch}.pth", weights_only=True
        )
        model.load_state_dict(checkpoint["model"])
        optimizer.load_state_dict(checkpoint["optimizer"])
        scheduler.load_state_dict(checkpoint["scheduler"])

        for _ in range(start_epoch):
            for _ in dataloader["train"]:
                break

        print(f"Resuming training from epoch {start_epoch}")

    print("Start training with", str(device).upper())
    for epoch in range(start_epoch, num_epochs):
        df_row = {"epoch": epoch + 1}

        for phase in ["train", "valid"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            with tqdm(
                total=len(dataloader[phase]),
                desc=f"{phase.capitalize()} epoch {epoch+1}/{num_epochs}",
                unit="bat",
            ) as pbar:
                for inputs, labels in dataloader[phase]:
                    inputs, labels = inputs.to(device), labels.to(device)

                    optimizer.zero_grad()
                    outputs = model(inputs)
                    _, predicts = torch.max(outputs.data, 1)
                    loss = criterion(outputs, labels)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                    running_loss += loss.item()
                    running_corrects += torch.sum(predicts == labels.data)

                    pbar.update()

                epoch_loss = running_loss / len(dataloader[phase])
                epoch_acc = 100 * running_corrects / len(dataloader[phase].dataset)
                pbar.set_postfix(
                    {
                        "loss": f"{epoch_loss:.4f}",
                        "acc": f"{epoch_acc:.2f}",
                    }
                )

                df_row[f"{phase}_loss"] = epoch_loss
                df_row[f"{phase}_acc"] = epoch_acc

                if phase == "valid" and epoch_acc > best_acc:
                    best_acc = epoch_acc

                    torch.save(
                        {
                            "model": model.state_dict(),
                            "optimizer": optimizer.state_dict(),
                            "scheduler": scheduler.state_dict(),
                        },
                        f"{save_path}/resnet_{epoch+1}.pth",
                    )

        scheduler.step()
        results.append(df_row)
        pd.DataFrame(results).to_csv(f"{save_path}/results.csv", index=False)

In [ ]:
def test(model: nn.Module, test_loader: DataLoader, weights_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    weights = torch.load(weights_path, weights_only=True)
    model.load_state_dict(weights["model"])

    correct = 0
    with tqdm(total=len(test_loader), desc="Test", unit="bat") as pbar:
        with torch.inference_mode():
            model.eval()
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicts = torch.max(outputs.data, 1)
                correct += (predicts == labels).sum().item()
                pbar.update()

    accuracy = 100 * correct / len(test_loader.dataset)
    print(f"Test accuracy: {accuracy:.2f}%")

In [7]:
def load_model(num_classes=10):
    model = resnet101(weights=ResNet101_Weights.DEFAULT)
    model.fc = nn.Sequential(
        nn.Linear(in_features=model.fc.in_features, out_features=256),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(in_features=256, out_features=num_classes, bias=False),
    )

    return model

# Train


In [ ]:
model = load_model(num_classes=10)

In [ ]:
dataloader = load_dataset("/kaggle/input/categories-classification/data", batch_size=32)

In [ ]:
# train(model, dataloader, num_epochs=20, resume=False)

In [ ]:
test(model, dataloader["test"], "./results/resnet_20.pth")